In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
import nfl_data_py as nfl
import datetime as dt

In [2]:
data_df = nfl.import_pbp_data([2021,2022])
players_df = nfl.import_rosters([2021,2022])
teams_df = nfl.import_team_desc
passing_stat_df = nfl.import_ngs_data('passing')
rushing_stat_df = nfl.import_ngs_data('rushing')
receiving_stat_df = nfl.import_ngs_data('receiving')

2021 done.
2022 done.
Downcasting floats.


In [48]:
data_df

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,offense_formation,offense_personnel,defenders_in_box,defense_personnel,number_of_pass_rushers,players_on_play,offense_players,defense_players,n_offense,n_defense
0,1.0,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,None,None,None,...,None,None,NaN,None,NaN,,,,0.0,0.0
1,40.0,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,TEN,home,ARI,...,None,None,NaN,None,NaN,43616;43680;52416;49410;53634;43556;53639;4625...,00-0032560;00-0036356;00-0035705;00-0036896;00...,00-0032496;00-0036652;00-0032355;00-0033455;00...,11.0,11.0
2,55.0,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,TEN,home,ARI,...,SINGLEBACK,"1 RB, 2 TE, 2 WR",7.0,"3 DL, 4 LB, 4 DB",NaN,52416;49410;38629;53445;43334;47816;47848;3853...,00-0029413;00-0032764;00-0029701;00-0027648;00...,00-0036356;00-0035705;00-0036933;00-0035236;00...,11.0,11.0
3,76.0,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,TEN,home,ARI,...,SHOTGUN,"1 RB, 2 TE, 2 WR",7.0,"3 DL, 4 LB, 4 DB",4.0,52416;49410;38629;53445;43334;47816;47848;3853...,00-0029413;00-0032764;00-0029701;00-0027648;00...,00-0036356;00-0035705;00-0036933;00-0035236;00...,11.0,11.0
4,100.0,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,TEN,home,ARI,...,SHOTGUN,"1 RB, 1 TE, 3 WR",5.0,"2 DL, 4 LB, 5 DB",4.0,52416;42401;49410;43556;38629;40008;47816;4784...,00-0032355;00-0029413;00-0029701;00-0033455;00...,00-0036356;00-0032127;00-0035705;00-0030528;00...,11.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56380,3915.0,2022_02_WAS_DET,2022091802,DET,WAS,REG,2,WAS,away,DET,...,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 4 LB, 5 DB",4.0,54467;41475;46151;53480;41321;48458;43722;4791...,00-0031095;00-0036618;00-0031362;00-0032602;00...,00-0037236;00-0034745;00-0035330;00-0035265;00...,11.0,11.0
56381,3940.0,2022_02_WAS_DET,2022091802,DET,WAS,REG,2,WAS,away,DET,...,SHOTGUN,"1 RB, 1 TE, 3 WR",5.0,"2 DL, 4 LB, 5 DB",6.0,44834;41475;54467;46151;53480;41321;47914;4372...,00-0031095;00-0036618;00-0031362;00-0035263;00...,00-0033880;00-0037236;00-0034745;00-0035265;00...,11.0,11.0
56382,3959.0,2022_02_WAS_DET,2022091802,DET,WAS,REG,2,DET,home,WAS,...,None,"1 RB, 3 TE, 1 WR",7.0,"5 DL, 2 LB, 4 DB",NaN,47809;46082;48707;53959;43305;53739;43373;5473...,00-0035567;00-0036754;00-0032381;00-0036534;00...,00-0035647;00-0034333;00-0033084;00-0037046;00...,11.0,11.0
56383,3980.0,2022_02_WAS_DET,2022091802,DET,WAS,REG,2,DET,home,WAS,...,None,"1 RB, 3 TE, 1 WR",7.0,"5 DL, 2 LB, 4 DB",NaN,47809;46082;48707;53959;43305;53739;43373;5473...,00-0035567;00-0036754;00-0032381;00-0036534;00...,00-0035647;00-0034333;00-0033084;00-0037046;00...,11.0,11.0


In [35]:
def dynamic_window_ewma(x):
    """
    Calculate rolling exponentially weighted EPA with a dynamic window size
    """
    values = np.zeros(len(x))
    for i, (_, row) in enumerate(x.iterrows()):
        epa = x.epa_shifted[:i+1]
        if row.week > 20:
            values[i] = epa.ewm(min_periods=1, span=row.week).mean().values[-1]
        else:
            values[i] = epa.ewm(min_periods=1, span=20).mean().values[-1]
            
    return pd.Series(values, index=x.index)

In [36]:
offense_epa = data_df.groupby(['posteam', 'season', 'week'], as_index=False)['epa'].mean()
defense_epa = data_df.groupby(['defteam', 'season', 'week'], as_index=False)['epa'].mean()
offense_epa['epa_shifted'] = offense_epa.groupby('posteam')['epa'].shift()
defense_epa['epa_shifted'] = defense_epa.groupby('defteam')['epa'].shift()

In [37]:
offense_epa['ewma_dynamic_window'] = offense_epa.groupby('posteam')\
.apply(dynamic_window_ewma).values

defense_epa['ewma_dynamic_window'] = defense_epa.groupby('defteam')['epa'].shift()

In [38]:
offense_epa = offense_epa.rename(columns={'posteam': 'team'})

In [39]:
defense_epa = defense_epa.rename(columns={'defteam': 'team'})

In [40]:
defense_epa

,team,season,week,epa,epa_shifted,ewma_dynamic_window
0,ARI,2021,1,-0.317720,NaN,NaN
1,ARI,2021,2,0.102555,-0.317720,-0.317720
2,ARI,2021,3,-0.175874,0.102555,0.102555
3,ARI,2021,4,-0.031375,-0.175874,-0.175874
4,ARI,2021,5,-0.152187,-0.031375,-0.031375
...,...,...,...,...,...,...
629,WAS,2021,16,0.294106,0.077146,0.077146
630,WAS,2021,17,0.110432,0.294106,0.294106
631,WAS,2021,18,-0.321945,0.110432,0.110432
632,WAS,2022,1,0.040248,-0.321945,-0.321945


In [41]:
epa = offense_epa.merge(defense_epa, on=['team', 'season', 'week'], suffixes=('_offense', '_defense'))

In [42]:
epa

,team,season,week,epa_offense,epa_shifted_offense,ewma_dynamic_window_offense,epa_defense,epa_shifted_defense,ewma_dynamic_window_defense
0,ARI,2021,1,0.080241,NaN,NaN,-0.317720,NaN,NaN
1,ARI,2021,2,0.084249,0.080241,0.080241,0.102555,-0.317720,-0.317720
2,ARI,2021,3,0.038362,0.084249,0.082346,-0.175874,0.102555,0.102555
3,ARI,2021,4,0.218309,0.038362,0.066195,-0.031375,-0.175874,-0.175874
4,ARI,2021,5,-0.061379,0.218309,0.110108,-0.152187,-0.031375,-0.031375
...,...,...,...,...,...,...,...,...,...
629,WAS,2021,16,-0.262447,-0.061564,-0.041595,0.294106,0.077146,0.077146
630,WAS,2021,17,0.020807,-0.262447,-0.068660,0.110432,0.294106,0.294106
631,WAS,2021,18,-0.069262,0.020807,-0.057988,-0.321945,0.110432,0.110432
632,WAS,2022,1,0.070693,-0.069262,-0.059301,0.040248,-0.321945,-0.321945


In [43]:
schedule = data_df[['season', 'week', 'home_team', 'away_team', 'home_score', 'away_score']]\
.drop_duplicates().reset_index(drop=True)\
.assign(home_team_win = lambda x: (x.home_score > x.away_score).astype(int))

df = schedule.merge(epa.rename(columns={'team': 'home_team'}), on=['home_team', 'season', 'week'])\
.merge(epa.rename(columns={'team': 'away_team'}), on=['away_team', 'season', 'week'], suffixes=('_home', '_away'))

df.tail(32)

,season,week,home_team,away_team,home_score,away_score,home_team_win,epa_offense_home,epa_shifted_offense_home,ewma_dynamic_window_offense_home,epa_defense_home,epa_shifted_defense_home,ewma_dynamic_window_defense_home,epa_offense_away,epa_shifted_offense_away,ewma_dynamic_window_offense_away,epa_defense_away,epa_shifted_defense_away,ewma_dynamic_window_defense_away
285,2022,1,NYJ,BAL,9,24,0,-0.195390,-0.365124,-0.069526,0.016643,-0.053699,-0.053699,0.016643,-0.185187,-0.010610,-0.195390,-0.100223,-0.100223
286,2022,1,LA,BUF,10,31,0,-0.232365,-0.051947,0.012065,0.137512,-0.106546,-0.106546,0.137512,0.261345,0.123612,-0.232365,0.291872,0.291872
287,2022,1,CAR,CLE,24,26,0,0.023642,-0.134302,-0.155780,0.081044,0.175102,0.175102,0.081044,-0.061541,-0.051187,0.023642,-0.123739,-0.123739
288,2022,1,SEA,DEN,17,16,1,0.071675,0.112876,0.026812,0.057772,-0.023545,-0.023545,0.057772,0.085916,-0.004300,0.071675,0.138022,0.138022
289,2022,1,MIN,GB,23,7,1,0.081753,-0.056427,-0.031607,-0.170877,-0.178721,-0.178721,-0.170877,-0.199412,0.065487,0.081753,-0.167644,-0.167644
290,2022,1,HOU,IND,20,20,0,-0.062144,0.107836,-0.102559,-0.020282,0.136311,0.136311,-0.020282,-0.206351,0.014832,-0.062144,0.048977,0.048977
291,2022,1,WAS,JAX,28,22,1,0.070693,-0.069262,-0.059301,0.040248,-0.321945,-0.321945,0.040248,0.048977,-0.100355,0.070693,-0.206351,-0.206351
292,2022,1,ARI,KC,21,44,0,0.011699,-0.331397,-0.035727,0.365711,0.064080,0.064080,0.365711,0.016382,0.134698,0.011699,0.098925,0.098925
293,2022,1,LAC,LV,24,19,1,-0.038851,0.086738,0.052070,-0.082090,0.139260,0.139260,-0.082090,0.014284,-0.012645,-0.038851,0.102302,0.102302
294,2022,1,MIA,NE,20,7,1,0.028838,0.102122,-0.063076,-0.162841,-0.007914,-0.007914,-0.162841,0.029039,0.054501,0.028838,0.529671,0.529671


In [44]:
df = df.drop(['epa_offense_away', 'epa_shifted_offense_away', 'epa_defense_away', 'epa_shifted_defense_away'], axis=1)

In [45]:
df.dropna()

,season,week,home_team,away_team,home_score,away_score,home_team_win,epa_offense_home,epa_shifted_offense_home,ewma_dynamic_window_offense_home,epa_defense_home,epa_shifted_defense_home,ewma_dynamic_window_defense_home,ewma_dynamic_window_offense_away,ewma_dynamic_window_defense_away
16,2021,2,TB,ATL,48,25,1,0.024064,0.060180,0.060180,-0.252526,0.066160,0.066160,-0.234793,0.094059
17,2021,2,MIA,BUF,0,35,0,-0.371971,0.006106,0.006106,0.129429,-0.003187,-0.003187,-0.124135,-0.006185
18,2021,2,CHI,CIN,20,17,1,-0.196814,-0.106607,-0.106607,-0.291214,0.222007,0.222007,0.023605,0.017370
19,2021,2,LAC,DAL,17,20,0,0.022971,0.016890,0.016890,0.088205,-0.065740,-0.065740,0.066160,0.060180
20,2021,2,JAX,DEN,13,23,0,-0.067900,-0.101188,-0.101188,0.113926,0.088506,0.088506,0.239024,0.021725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,2022,2,CLE,NYJ,30,31,0,0.194344,0.081044,-0.036104,0.185281,0.023642,0.023642,-0.083882,0.016643
313,2022,2,SF,SEA,27,7,1,0.080772,-0.195610,-0.013754,-0.242324,-0.106012,-0.106012,0.031929,0.057772
314,2022,2,NO,TB,10,20,0,-0.327033,0.041013,-0.106389,-0.156965,0.011905,0.011905,0.041063,-0.293072
315,2022,2,BUF,TEN,41,7,1,0.199349,0.137512,0.125142,-0.274970,-0.232365,-0.232365,-0.007801,0.041248


In [51]:
df['home_line'] = (((df['ewma_dynamic_window_offense_home']) + (df['ewma_dynamic_window_defense_away'])) - ((df['ewma_dynamic_window_offense_away']) + (df['ewma_dynamic_window_defense_home'])))*-10

In [52]:
df.tail(32)

,season,week,home_team,away_team,home_score,away_score,home_team_win,epa_offense_home,epa_shifted_offense_home,ewma_dynamic_window_offense_home,epa_defense_home,epa_shifted_defense_home,ewma_dynamic_window_defense_home,ewma_dynamic_window_offense_away,ewma_dynamic_window_defense_away,home_line
285,2022,1,NYJ,BAL,9,24,0,-0.195390,-0.365124,-0.069526,0.016643,-0.053699,-0.053699,-0.010610,-0.100223,1.054392
286,2022,1,LA,BUF,10,31,0,-0.232365,-0.051947,0.012065,0.137512,-0.106546,-0.106546,0.123612,0.291872,-2.868712
287,2022,1,CAR,CLE,24,26,0,0.023642,-0.134302,-0.155780,0.081044,0.175102,0.175102,-0.051187,-0.123739,4.034341
288,2022,1,SEA,DEN,17,16,1,0.071675,0.112876,0.026812,0.057772,-0.023545,-0.023545,-0.004300,0.138022,-1.926794
289,2022,1,MIN,GB,23,7,1,0.081753,-0.056427,-0.031607,-0.170877,-0.178721,-0.178721,0.065487,-0.167644,0.860172
290,2022,1,HOU,IND,20,20,0,-0.062144,0.107836,-0.102559,-0.020282,0.136311,0.136311,0.014832,0.048977,2.047250
291,2022,1,WAS,JAX,28,22,1,0.070693,-0.069262,-0.059301,0.040248,-0.321945,-0.321945,-0.100355,-0.206351,-1.566479
292,2022,1,ARI,KC,21,44,0,0.011699,-0.331397,-0.035727,0.365711,0.064080,0.064080,0.134698,0.098925,1.355801
293,2022,1,LAC,LV,24,19,1,-0.038851,0.086738,0.052070,-0.082090,0.139260,0.139260,-0.012645,0.102302,-0.277567
294,2022,1,MIA,NE,20,7,1,0.028838,0.102122,-0.063076,-0.162841,-0.007914,-0.007914,0.054501,0.529671,-4.200065
